# Setup Bucket for Multicloud E2E RAG demo

## Create Bucket for documents in Object Storage

In [ ]:
!pip install "minio<7.0" -q

In [ ]:
import os

from minio import Minio
from minio.error import BucketAlreadyOwnedByYou, NoSuchKey

In [ ]:
MINIO_HOST = os.environ["MINIO_ENDPOINT_URL"].split("http://")[1]

# Initialize a MinIO client
mc = Minio(
    endpoint=MINIO_HOST,
    access_key=os.environ["AWS_ACCESS_KEY_ID"],
    secret_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    secure=False,
)

In [ ]:
BUCKET = "rag-demo-source"
FORCE_RECREATE = True

if FORCE_RECREATE:
    #empty the bucket
    for obj in mc.list_objects(BUCKET, recursive=True):
        mc.remove_object(BUCKET, obj.object_name)
    mc.remove_bucket(BUCKET)

try:
    mc.make_bucket(BUCKET)
    print(f"Bucket {BUCKET} created!")
except BucketAlreadyOwnedByYou:
    print(f"Bucket {BUCKET} already exists!")

In [ ]:
# List buckets
buckets = mc.list_buckets()
for bucket in buckets:
    print(bucket.name)
    # List objects in bucket
    objects = mc.list_objects(bucket.name)
    for obj in objects:
        print("\t", obj.object_name)

## Upload files from "documents" to the bucket

In [ ]:
DOCUMENT_FOLDER = "./../documents/"
for f in os.listdir(DOCUMENT_FOLDER):
    if os.path.isfile(f"{DOCUMENT_FOLDER}/{f}"):
        mc.fput_object(BUCKET, f, DOCUMENT_FOLDER+f)
        print("Uploaded file:", f)